In [56]:
import numpy as np
import pandas as pd
from pyecharts.charts import WordCloud, Bar

In [57]:
file1=r'D:\Programing\python_projects\data_analysis_basics\data_set\rating.txt'
rating=pd.read_csv(file1, header=None, names=['user_id', 'book_id', 'rating'])
file2=r'D:\Programing\python_projects\data_analysis_basics\data_set\to_read.csv'
to_read=pd.read_csv(file2)
file3=r'D:\Programing\python_projects\data_analysis_basics\data_set\books.csv'
books=pd.read_csv(file3)
file4=r'D:\Programing\python_projects\data_analysis_basics\data_set\book_tags.csv'
book_tags=pd.read_csv(file4)

In [59]:
# 创建空DF存放书本平均得分，并指定索引名
book_df=pd.DataFrame({
    'rating_sum':[],
    'rating_counts':[]
})
book_df.index.name='book_id'

In [60]:
# 计算不同的书被评分的次数
book_df['rating_counts']=rating.groupby('book_id')['user_id'].count()
book_df.head()

,rating_sum,rating_counts
book_id,,
1,NaN,22806
2,NaN,21850
3,NaN,16931
4,NaN,19088
5,NaN,16604


In [61]:
# 计算不同的书被评分的总和
book_df['rating_sum']=rating.groupby('book_id')['rating'].sum()
book_df.head()

,rating_sum,rating_counts
book_id,,
1,97603,22806
2,95077,21850
3,54422,16931
4,82639,19088
5,62634,16604


In [63]:
# 计算不同书的平均得分
book_df['avg_rating']=book_df['rating_sum']/book_df['rating_counts']
book_df.head()


,rating_sum,rating_counts,avg_rating
book_id,,,
1,97603,22806,4.279707
2,95077,21850,4.351350
3,54422,16931,3.214341
4,82639,19088,4.329369
5,62634,16604,3.772224
